In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d niharika41298/yoga-poses-dataset

 99% 284M/288M [00:04<00:00, 43.9MB/s]
100% 288M/288M [00:04<00:00, 62.7MB/s]


In [4]:
import zipfile
zip_refee=zipfile.ZipFile('/content/yoga-poses-dataset.zip','r')
zip_refee.extractall('/content')
zip_refee.close()

# Data Augumentation

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers.schedules import ExponentialDecay

In [6]:
train='/content/DATASET/TRAIN'
test='/content/DATASET/TEST'

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen=ImageDataGenerator(rescale=1./255.)


In [8]:
img_size=(255,255)
batch_size=32
train_generator = train_datagen.flow_from_directory(train,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

Found 1081 images belonging to 5 classes.
Found 470 images belonging to 5 classes.


# Model


In [9]:
lr_schedual=ExponentialDecay(initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9)

In [10]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2,padding='valid'))


model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2,padding='valid'))

model.add(Conv2D(256,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2,padding='valid'))

model.add(Conv2D(512,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2,padding='valid'))


model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 254, 254, 64)      256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 128)     73856     
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 128)     512       
 chNormalization)                                                
                                                        

In [43]:
# model=keras.saving.load_model('/content/best_model.h5')/# to train model again

In [44]:
model.compile(optimizer= Adam(learning_rate=lr_schedual),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

In [46]:
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=test_generator,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
34/34 [==============================] - 61s 2s/step - loss: 0.7349 - accuracy: 0.7345 - val_loss: 0.6725 - val_accuracy: 0.7447 - lr: 9.9653e-04
Epoch 2/50
34/34 [==============================] - 97s 3s/step - loss: 0.7625 - accuracy: 0.7151 - val_loss: 0.4120 - val_accuracy: 0.8404 - lr: 9.9297e-04
Epoch 3/50
34/34 [==============================] - 38s 1s/step - loss: 0.8601 - accuracy: 0.6892 - val_loss: 2.6090 - val_accuracy: 0.3511 - lr: 9.8941e-04
Epoch 4/50
34/34 [==============================] - 37s 1s/step - loss: 0.8120 - accuracy: 0.7216 - val_loss: 0.8900 - val_accuracy: 0.6723 - lr: 9.8588e-04
Epoch 5/50
34/34 [==============================] - 39s 1s/step - loss: 0.7903 - accuracy: 0.7142 - val_loss: 0.4375 - val_accuracy: 0.8404 - lr: 9.8235e-04
Epoch 6/50
34/34 [==============================] - 41s 1s/step - loss: 0.7168 - accuracy: 0.7438 - val_loss: 0.4128 - val_accuracy: 0.8511 - lr: 9.7884e-04
Epoch 7/50
34/34 [==============================] - 41s 1s

# test data

In [50]:
import cv2
image = cv2.imread("/content/DATASET/TEST/goddess/00000000.jpg")
image = cv2.resize(image, (256, 256))  # Adjust the size based on your model requirements
image = image / 255.0  # Normalize pixel values
image=np.expand_dims(image, axis=0)

In [52]:
class_labels = ['downdog', 'goddess', 'plank', 'tree','warrior2']
predictions = model.predict(image)
predicted_class_index = np.argmax(predictions)
predicted_class = class_labels[predicted_class_index]
print(predicted_class)

1/1 [==============================] - 0s 25ms/step
plank
